# DEEP LEARNING _ FINAL PROJECT (PART 1)

> ## Members: 
> <p style="text-align: left;"><font color = blue > Amanuel Abrdo Tereda</p>
> <p style="text-align: left;"><font color = blue > Stefalo Acha</p>

***
> *Date: May 02, 2023*
> ### <p style="text-align: right;">Instructor: <font color = blue > Dr. Letu Qingge</p>
***



## Image classification with different Neural Netowrk setups

> **1. 100 ReLU hidden units, 5 Sigmoid output units, MeanSquaredError loss**

> **2.	100 Tanh hidden units, 5 Sigmoid output units, MeanSquaredError loss**

> **3.	100 Sigmoid hidden units, 5 Sigmoid output units, MeanSquaredError loss**

> **4.	100 Sigmoid hidden units, 5 Linear output units, Softmax loss**

> **5.	100 Tanh hidden units, 5 Linear output units, Softmax loss**

        - Without an optimizer and an initialization
    
        - With an optimizer of momentum = 0.9, weight decay = 0.02, and 'glorot' initialization.

        - With an optimizer of momentum = 0.9, weight decay = 0.02, and 'random' initialization.

***

In [1]:
!git clone https://github.com/Amumu-ze1ast/Deep_Learning_Final_try.git

Cloning into 'Deep_Learning_Final_try'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), 5.09 MiB | 4.33 MiB/s, done.


In [2]:
import zipfile
with zipfile.ZipFile("/content/Deep_Learning_Final_try/lincoln.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [3]:
import lincoln
from lincoln.layers import Dense
from lincoln.losses import SoftmaxCrossEntropy, MeanSquaredError
from lincoln.optimizers import Optimizer, SGD, SGDMomentum
from lincoln.activations import Sigmoid, Tanh, Linear, ReLU
from lincoln.network import NeuralNetwork
from lincoln.train import Trainer
from lincoln.utils import mnist
from lincoln.utils.np_utils import softmax

In [4]:
import numpy as np
import cv2
import pandas as pd
import PIL.Image as Image
import os
import matplotlib.pylab as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


In [5]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)

228813984/228813984 [==============================] - 2s 0us/step


In [6]:
import pathlib
data_dir = pathlib.Path(data_dir)
list_images = list(data_dir.glob('*/*.jpg'))
image_count = len(list_images)

In [7]:
#Make up python dictionary

flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

#Creating a label directory

flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

class_names = list(flowers_labels_dict.keys())

In [8]:
# Resize all images using for loop

IMAGE_SHAPE = (224,224)
#IMAGE_SHAPE = (28,28)

x, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized_img = cv2.resize(img_gray,(IMAGE_SHAPE)) #IMAGE_SHAPE = (224,224)
        
        #resized_img = cv2.resize(img,(IMAGE_SHAPE)) #IMAGE_SHAPE = (224,224)
        
        x.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [9]:
x[0].shape

(224, 224)

In [10]:
x = np.array(x)
y = np.array(y)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3, random_state=25)

In [12]:
x_train, x_test = x_train - np.mean(x_train), x_test - np.mean(x_train)

In [13]:
np.min(x_train), np.max(x_train), np.min(x_test), np.max(x_test)

(-108.02989123420474,
 146.97010876579526,
 -108.02989123420474,
 146.97010876579526)

In [14]:
x_train, x_test = x_train / np.std(x_train), x_test / np.std(x_train)

In [15]:
np.min(x_train), np.max(x_train), np.min(x_test), np.max(x_test)

(-1.6288192195108047,
 2.215939821066163,
 -1.6288192195108047,
 2.215939821066163)

In [16]:
#verify
print("shape of input  - training set =", x_train.shape)
print("shape of output - training set =", y_train.shape)
print("shape of input  - testing set  =", x_test.shape)
print("shape of output - testing set  =", y_test.shape)

shape of input  - training set = (2446, 224, 224)
shape of output - training set = (2446,)
shape of input  - testing set  = (1224, 224, 224)
shape of output - testing set  = (1224,)


In [17]:
x_train.shape[1]

224

In [18]:
x_train = x_train.reshape(len(x_train),(x_train.shape[1])*(x_train.shape[1]))
x_test  = x_test.reshape(len(x_test),(x_test.shape[1])*(x_test.shape[1]))

In [19]:
#verify
print("shape of input  - training set =", x_train.shape)
print("shape of output - training set =", y_train.shape)
print("shape of input  - testing set  =", x_test.shape)
print("shape of output - testing set  =", y_test.shape)

shape of input  - training set = (2446, 50176)
shape of output - training set = (2446,)
shape of input  - testing set  = (1224, 50176)
shape of output - testing set  = (1224,)


In [20]:
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 5))
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 5))
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

In [21]:
#verify
print("shape of input  - training set    =", x_train.shape)
print("shape of output - training lables =", train_labels.shape)
print("shape of input  - testing set     =", x_test.shape)
print("shape of output - test labels     =", test_labels.shape)

shape of input  - training set    = (2446, 50176)
shape of output - training lables = (2446, 5)
shape of input  - testing set     = (1224, 50176)
shape of output - test labels     = (1224, 5)


In [22]:
def calc_accuracy_model(model, test_set):
    return print(f'''The model validation accuracy is: 
    {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

### a) 100 ReLU hidden units, 5 Sigmoid output units, MeanSquaredError loss

In [23]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=ReLU()),
            Dense(neurons=5, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=False), 
seed=20190119)

print('100 ReLU hidden units, 5 Sigmoid output units, MeanSquaredError loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 ReLU hidden units, 5 Sigmoid output units, MeanSquaredError loss
_________________________________/\____________________________________


/content/lincoln/activations.py:28: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-1.0 * self.input_))


Validation loss after 10 epochs is 1.388
Validation loss after 20 epochs is 1.320
Validation loss after 30 epochs is 1.156
Validation loss after 40 epochs is 1.047
Validation loss after 50 epochs is 1.023
Validation loss after 60 epochs is 1.006
Validation loss after 70 epochs is 1.006

_________________________________\/____________________________________
The model validation accuracy is: 
    19.61%


### b) 100 Tanh hidden units, 5 Sigmoid output units, MeanSquaredError loss 

In [24]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Tanh()),
            Dense(neurons=5, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=False), 
seed=20190119)

print('100 Tanh hidden units, 5 Sigmoid output units, MeanSquaredError loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Tanh hidden units, 5 Sigmoid output units, MeanSquaredError loss
_________________________________/\____________________________________
Validation loss after 10 epochs is 1.806
Validation loss after 20 epochs is 1.685
Validation loss after 30 epochs is 1.609
Validation loss after 40 epochs is 1.558
Validation loss after 50 epochs is 1.529
Validation loss after 60 epochs is 1.500
Validation loss after 70 epochs is 1.474

_________________________________\/____________________________________
The model validation accuracy is: 
    25.33%


### c) 100 Sigmoid hidden units, 5 Sigmoid output units, MeanSquaredError loss

In [25]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Sigmoid()),
            Dense(neurons=5, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=False), 
seed=20190119)

print('100 Sigmoid hidden units, 5 Sigmoid output units, MeanSquaredError loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Sigmoid hidden units, 5 Sigmoid output units, MeanSquaredError loss
_________________________________/\____________________________________
Validation loss after 10 epochs is 1.056
Validation loss after 20 epochs is 1.030
Validation loss after 30 epochs is 1.029

Loss increased after epoch 40, final loss was 1.029, 
using the model from epoch 30

_________________________________\/____________________________________
The model validation accuracy is: 
    23.53%


### d) 100 Sigmoid hidden units, 5 Linear output units, Softmax loss 

In [26]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Sigmoid()),
            Dense(neurons=5, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

print('100 Sigmoid hidden units, 5 Linear output units, Softmax loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 10,
            eval_every = 1,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Sigmoid hidden units, 5 Linear output units, Softmax loss
_________________________________/\____________________________________
Validation loss after 1 epochs is 7.092
Validation loss after 2 epochs is 6.569
Validation loss after 3 epochs is 6.223
Validation loss after 4 epochs is 6.047
Validation loss after 5 epochs is 5.789
Validation loss after 6 epochs is 5.622
Validation loss after 7 epochs is 5.434
Validation loss after 8 epochs is 5.266
Validation loss after 9 epochs is 5.171
Validation loss after 10 epochs is 5.044

_________________________________\/____________________________________
The model validation accuracy is: 
    24.43%


### e) 100 Tanh hidden units, 5 Linear output units, Softmax loss

In [27]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Tanh()),
            Dense(neurons=5, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

print('100 Tanh hidden units, 5 Linear output units, Softmax loss')
print('_________________________________/\____________________________________')

trainer = Trainer(model, SGD(0.1))
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Tanh hidden units, 5 Linear output units, Softmax loss
_________________________________/\____________________________________
Validation loss after 10 epochs is 5.718
Validation loss after 20 epochs is 3.559
Validation loss after 30 epochs is 3.101
Validation loss after 40 epochs is 3.000
Validation loss after 50 epochs is 2.985

Loss increased after epoch 60, final loss was 2.985, 
using the model from epoch 50

_________________________________\/____________________________________
The model validation accuracy is: 
    27.37%


## Comparing the weight initialization algorithms for the best model

### a) Random

In [28]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Tanh(),
                  weight_init="random"),
            Dense(neurons=5, 
                  activation=Linear(),
                  weight_init="random")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optimizer = SGDMomentum(0.25, 
                        momentum=0.9, 
                        final_lr = 0.02, 
                        decay_type='exponential')

print('100 Tanh hidden units, 5 Linear output units, Softmax loss')
print("With an optimizer of momentum = 0.9, weight decay = 0.02, and 'random' initialization")
print('_________________________________/\____________________________________')

trainer = Trainer(model, optimizer)
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
calc_accuracy_model(model, x_test)

100 Tanh hidden units, 5 Linear output units, Softmax loss
With an optimizer of momentum = 0.9, weight decay = 0.02, and 'random' initialization
_________________________________/\____________________________________
Validation loss after 10 epochs is 3.601

Loss increased after epoch 20, final loss was 3.601, 
using the model from epoch 10

The model validation accuracy is: 
    26.80%


### b) Glorot 

In [29]:
model = NeuralNetwork(
    layers=[Dense(neurons=100, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=5, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optimizer = SGDMomentum(0.25, 
                        momentum=0.9, 
                        final_lr = 0.02, 
                        decay_type='exponential')

print('100 Tanh hidden units, 5 Linear output units, Softmax loss')
print("With an optimizer of momentum = 0.9, weight decay = 0.02, and 'glorot' initialization")
print('_________________________________/\____________________________________')

trainer = Trainer(model, optimizer)
trainer.fit(x_train, train_labels, x_test, test_labels,
            epochs = 70,
            eval_every = 10,
            seed=20190119,
            batch_size=70);
print()
print('_________________________________\/____________________________________')
calc_accuracy_model(model, x_test)

100 Tanh hidden units, 5 Linear output units, Softmax loss
With an optimizer of momentum = 0.9, weight decay = 0.02, and 'glorot' initialization
_________________________________/\____________________________________
Validation loss after 10 epochs is 3.796
Validation loss after 20 epochs is 3.604
Validation loss after 30 epochs is 3.082
Validation loss after 40 epochs is 2.931

Loss increased after epoch 50, final loss was 2.931, 
using the model from epoch 40

_________________________________\/____________________________________
The model validation accuracy is: 
    28.35%
